In [15]:
from __future__ import print_function
import cv2 as cv
import argparse
import numpy as np

def detectAndDisplay(frame):
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame_gray = cv.equalizeHist(frame_gray)

    #-- Detect faces
    faces = face_cascade.detectMultiScale(frame_gray)
    if len(faces) == 0:  # Add this check to see if any faces were detected
        return None
    for (x,y,w,h) in faces:
        center = (x + w//2, y + h//2)
        frame = cv.ellipse(frame, center, (w//2, h//2), 0, 0, 360, (255, 0, 255), 4)
        faceROI = frame_gray[y:y+h,x:x+w]
        #-- In each face, detect eyes
        eyes = eyes_cascade.detectMultiScale(faceROI)
        for (x2,y2,w2,h2) in eyes:
            eye_center = (x + x2 + w2//2, y + y2 + h2//2)
            radius = int(round((w2 + h2)*0.25))
            frame = cv.circle(frame, eye_center, radius, (255, 0, 0 ), 4)
        
    # Create mask
    mask = np.zeros(frame.shape[:2], np.uint8)

    # Use the face as the object area
    for (x, y, w, h) in faces:
        mask[y:y+h, x:x+w] = 1

    # Use the area above the face 
    if len(faces) > 0 and y > 1: 
        mask[0:y, :] = 2

    # Define the background and foreground model for GrabCut
    bgdModel, fgdModel  = np.zeros((1, 65), np.float64), np.zeros((1, 65), np.float64)

    # Apply GrabCut
    cv.grabCut(frame, mask, None, bgdModel, fgdModel, 5, cv.GC_INIT_WITH_MASK)
    mask2 = np.where((mask==2)|(mask==0), 0, 1).astype('uint8')
    frame = frame*mask2[:, :, np.newaxis]

    cv.imshow('Capture - Face detection', frame)

    return frame  

# Cascade code from documentation
parser = argparse.ArgumentParser(description='Code for Cascade Classifier tutorial.')
parser.add_argument('--face_cascade', help='Path to face cascade.', default='data/haarcascades/haarcascade_frontalface_alt.xml')
parser.add_argument('--eyes_cascade', help='Path to eyes cascade.', default='data/haarcascades/haarcascade_eye_tree_eyeglasses.xml')
parser.add_argument('--camera', help='Camera divide number.', type=int, default=0)
args = parser.parse_args()
face_cascade_name = args.face_cascade
eyes_cascade_name = args.eyes_cascade
face_cascade = cv.CascadeClassifier()
eyes_cascade = cv.CascadeClassifier()

#-- 1. Load the cascades
if not face_cascade.load('haarcascade_frontalface_default.xml'):
    print('--(!)Error loading face cascade')
    exit(0)
if not eyes_cascade.load('haarcascade_eye.xml'):
    print('--(!)Error loading eyes cascade')
    exit(0)


#-- 2. Read the video stream
camera_device = args.camera
cap = cv.VideoCapture(camera_device)
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

while True:
    ret, frame = cap.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break
    frame = cv.resize(frame, (0, 0), fx=0.5, fy=0.5)
    processed_frame = detectAndDisplay(frame)
    if processed_frame is not None:  # Only write the frame if a face was detected
        processed_frame = cv.resize(processed_frame, (processed_frame.shape[1]*2, processed_frame.shape[0]*2))
        out.write(processed_frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release 
out.release()
cap.release()  
cv.destroyAllWindows()